# BQ ML K-Means Examples

This notebook shows how K-means is being and can be used to find insights in the search data

In [34]:
from google.cloud import bigquery


PREDICTED_ON_DATE = '2021-06-06'
FORECAST_WEEKS = 30
project = 'cpg-cdp'
source_table = 'prqedict_c52_h52_fixed'

In [35]:
bq_client = bigquery.Client(project=project)

### Utilized in the pipeline (Can be changed)

In [36]:
bq_client.query(
      f"""
create table if not exists `{project}.trendspotting.cluster_train_{PREDICTED_ON_DATE}_{FORECAST_WEEKS}` as (
    with six_mo_val as (select *, predicted_category_rank.value as six_mo_forecast from `cpg-cdp.trendspotting.{source_table}` where predicted_on_date = '2021-06-06' and date = '2021-12-26')
    , three_mo_val as (select series_id, predicted_category_rank.value as three_mo_forecast from `cpg-cdp.trendspotting.{source_table}` where predicted_on_date = '2021-06-06' and date = '2021-09-26')
    , one_mo_val as (select series_id, predicted_category_rank.value as one_mo_forecast from `cpg-cdp.trendspotting.{source_table}` where predicted_on_date = '2021-06-06' and date = '2021-07-11')
    , actuals as (select distinct series_id, max(category_rank_int) as current_rank from `cpg-cdp.trendspotting.{source_table}` where predicted_on_date = '2021-06-06' and date = '2021-06-06' group by series_id)
    select a.* 
        except(predicted_category_rank, split_col,category_rank, category_rank_int, series_id), 
        b.three_mo_forecast, 
        c.one_mo_forecast, 
        d.current_rank
    from six_mo_val a
        inner join three_mo_val b on (a.series_id = b.series_id)  
        inner join one_mo_val c on (a.series_id = c.series_id)
        inner join actuals d on (a.series_id = d.series_id)
        )
          """).result()

In [ ]:
bq_client.query(
      f"""
CREATE OR REPLACE MODEL
  trendspotting.50_clusters_draft OPTIONS(model_type='kmeans',
  kmeans_init_method = 'KMEANS++',
  STANDARDIZE_FEATURES = True,
    num_clusters=50) AS (
  select * from (  
    with six_mo_val as (select *, predicted_category_rank.value as six_mo_forecast from `cpg-cdp.trendspotting.{source_table}` where predicted_on_date = '2021-06-06' and date = '2021-12-26')
    , three_mo_val as (select series_id, predicted_category_rank.value as three_mo_forecast from `cpg-cdp.trendspotting.{source_table}` where predicted_on_date = '2021-06-06' and date = '2021-09-26')
    , one_mo_val as (select series_id, predicted_category_rank.value as one_mo_forecast from `cpg-cdp.trendspotting.{source_table}` where predicted_on_date = '2021-06-06' and date = '2021-07-11')
    , actuals as (select distinct series_id, max(category_rank_int) as current_rank from `cpg-cdp.trendspotting.{source_table}` where predicted_on_date = '2021-06-06' and date = '2021-06-06' group by series_id)
    select a.* 
        except(date, predicted_on_date, predicted_category_rank, split_col,category_rank, category_rank_int, series_id), 
        b.three_mo_forecast, 
        c.one_mo_forecast, 
        d.current_rank,
        six_mo_forecast - d.current_rank as delta_rank
    from six_mo_val a
        inner join three_mo_val b on (a.series_id = b.series_id)  
        inner join one_mo_val c on (a.series_id = c.series_id)
        inner join actuals d on (a.series_id = d.series_id)
        )) order by delta_rank limit 500
          """).result()

### These queries filter results from the forecast to be used in k-means

In [21]:
bq_client.query(
      f"""
CREATE OR REPLACE MODEL
  trendspotting.50_clusters_draft_top500_filtered OPTIONS(model_type='kmeans',
  kmeans_init_method = 'KMEANS++',
    num_clusters=50) AS (
    
    with data as 
(
    SELECT term, avg(predicted_category_rank.value) as predicted_rank, avg(category_rank_int) as category_rank FROM `cpg-cdp.trendspotting.predict_c52_h52_fixed` 
    where predicted_on_date = 
    '2021-06-06' and date = '2021-12-26' group by term)

select * from data
order by predicted_rank desc LIMIT 500
        )
          """).result()

### This one looks at the difference between actual and predicted six months out

In [32]:
bq_client.query(
      f"""
CREATE OR REPLACE MODEL
  trendspotting.50_clusters_draft_top500_filtered_change OPTIONS(model_type='kmeans',
  kmeans_init_method = 'KMEANS++',
    num_clusters=50) AS (
    
    with data as 
(
    SELECT *, 
    category_rank_int - predicted_category_rank.value as six_mo_change
    FROM `cpg-cdp.trendspotting.predict_c52_h52_fixed` 
    where predicted_on_date = 
    '2021-06-06' and date = '2021-12-26')

select * from data
order by six_mo_change desc LIMIT 5000
        )
          """).result()

In [42]:
bq_client.query(
      f"""
CREATE OR REPLACE MODEL
  trendspotting.50_clusters_draft_top500_filtered_change OPTIONS(model_type='kmeans',
  kmeans_init_method = 'KMEANS++',
  STANDARDIZE_FEATURES = True,
    num_clusters=50) AS (
    
 with data as (select *, current_rank - six_mo_forecast as six_mo_delta
 from `{project}.trendspotting.cluster_train_{PREDICTED_ON_DATE}_{FORECAST_WEEKS}`
        )
    select * except(date, predicted_on_date) from data 
    order by six_mo_delta desc limit 500
    )
          """).result()

In [51]:
### Cosine distance use this

In [128]:
top_n_results = 30000 #???
n_clusters = 30 # ???
distance = "COSINE" #better for text
k_means_model_name = f"draft_clusters_{n_clusters}_{top_n_results}_{distance}_top500_filtered_RMSLE"
source_table = f'cpg-cdp.trendspotting.predict_c52_h52_fixed' #rmse table

bq_client.query(
  f"""
CREATE OR REPLACE MODEL
    trendspotting.{k_means_model_name} OPTIONS(model_type='kmeans',
    kmeans_init_method = 'KMEANS++',
    DISTANCE_TYPE = '{distance}',
    STANDARDIZE_FEATURES = TRUE,
    num_clusters={n_clusters}) AS (
    (select * from ( 
    with six_mo_val as (select *, predicted_category_rank.value as six_mo_forecast from `{source_table}` 
        where predicted_on_date = '2021-06-06' and date = '2021-12-26')
    , three_mo_val as (select series_id, predicted_category_rank.value as three_mo_forecast from `{source_table}` 
        where predicted_on_date = '2021-06-06' and date = '2021-09-26')
    , one_mo_val as (select series_id, predicted_category_rank.value as one_mo_forecast from `{source_table}` 
        where predicted_on_date = '2021-06-06' and date = '2021-07-11')
    , actuals as (select distinct series_id, max(category_rank_int) as current_rank from `{source_table}` 
        where predicted_on_date = '2021-06-06' and date = '2021-06-06' group by series_id)
    select a.*
        except(date, predicted_on_date, predicted_category_rank, split_col,category_rank, category_rank_int, series_id), 
        b.three_mo_forecast, 
        c.one_mo_forecast, 
        d.current_rank,
        d.current_rank - six_mo_forecast as rank_delta_6,
        d.current_rank - c.one_mo_forecast as rank_delta_1,
        d.current_rank - b.three_mo_forecast as rank_delta_3,
        case when current_rank >= 500 then 'bottom_500' else 'top_500' end as top_500_flag
    from six_mo_val a
        inner join three_mo_val b on (a.series_id = b.series_id)  
        inner join one_mo_val c on (a.series_id = c.series_id)
        inner join actuals d on (a.series_id = d.series_id)
        ) order by six_mo_forecast asc limit {top_n_results}))
        """).result()

In [116]:
print(f"{source_table}_n_clus_{n_clusters}")

cpg-cdp.trendspotting.predict_c52_h52_fixed_n_clus_30


In [91]:
# bq_client.query(
    
# f""" 
#  CREATE TABLE IF NOT EXISTS {source_table}_n_clus_{n_clusters} as (
#  select * from ( 
#     with six_mo_val as (select *, predicted_category_rank.value as six_mo_forecast from `{source_table}` 
#         where predicted_on_date = '2021-06-06' and date = '2021-12-26')
#     , three_mo_val as (select series_id, predicted_category_rank.value as three_mo_forecast from `{source_table}` 
#         where predicted_on_date = '2021-06-06' and date = '2021-09-26')
#     , one_mo_val as (select series_id, predicted_category_rank.value as one_mo_forecast from `{source_table}` 
#         where predicted_on_date = '2021-06-06' and date = '2021-07-11')
#     , actuals as (select distinct series_id, max(category_rank_int) as current_rank from `{source_table}` 
#         where predicted_on_date = '2021-06-06' and date = '2021-06-06' group by series_id)
#     select a.* 
#         except(date, predicted_on_date, predicted_category_rank, split_col,category_rank, category_rank_int, series_id), 
#         b.three_mo_forecast, 
#         c.one_mo_forecast, 
#         d.current_rank,
#         six_mo_forecast - d.current_rank as rank_delta_6,
#         c.one_mo_forecast - d.current_rank as rank_delta_1,
#         b.three_mo_forecast - d.current_rank as rank_detla_3
#     from six_mo_val a
#         inner join three_mo_val b on (a.series_id = b.series_id)  
#         inner join one_mo_val c on (a.series_id = c.series_id)
#         inner join actuals d on (a.series_id = d.series_id)
#         )) order by rank_delta_6 desc limit {top_n_results}
#      """
# ).result()

In [129]:
bq_client.query( 
f""" 
 CREATE TABLE IF NOT EXISTS {source_table}_n_clus_{n_clusters}_scored_rmsle as (
 SELECT * 
 FROM 
 ML.PREDICT( MODEL trendspotting.{k_means_model_name}, 
 (select * from (select * from ( 
    with six_mo_val as (select *, predicted_category_rank.value as six_mo_forecast from `{source_table}` 
        where predicted_on_date = '2021-06-06' and date = '2021-12-26')
    , three_mo_val as (select series_id, predicted_category_rank.value as three_mo_forecast from `{source_table}` 
        where predicted_on_date = '2021-06-06' and date = '2021-09-26')
    , one_mo_val as (select series_id, predicted_category_rank.value as one_mo_forecast from `{source_table}` 
        where predicted_on_date = '2021-06-06' and date = '2021-07-11')
    , actuals as (select distinct series_id, max(category_rank_int) as current_rank from `{source_table}` 
        where predicted_on_date = '2021-06-06' and date = '2021-06-06' group by series_id)
    select a.*
        except(date, predicted_on_date, predicted_category_rank, split_col,category_rank, category_rank_int, series_id), 
        b.three_mo_forecast, 
        c.one_mo_forecast, 
        d.current_rank,
        d.current_rank - six_mo_forecast as rank_delta_6,
        d.current_rank - c.one_mo_forecast as rank_delta_1,
        d.current_rank - b.three_mo_forecast as rank_delta_3,
        case when current_rank >= 500 then 'bottom_500' else 'top_500' end as top_500_flag
    from six_mo_val a
        inner join three_mo_val b on (a.series_id = b.series_id)  
        inner join one_mo_val c on (a.series_id = c.series_id)
        inner join actuals d on (a.series_id = d.series_id)
        ) order by six_mo_forecast asc limit {top_n_results}))
 ))
     """
).result()

## Run clustering for the entire futurama dataset

##### n_clusters = 100 # max
distance = "COSINE" #better for text
k_means_model_name = f"term_only_20_21_{n_clusters}_{distance}"
source_table = f'cpg-cdp.trendspotting.futurama_weekly' 
top_n_results = 100_000

bq_client.query(
  f"""
CREATE OR REPLACE MODEL
    trendspotting.{k_means_model_name} OPTIONS(model_type='kmeans',
    kmeans_init_method = 'KMEANS++',
    DISTANCE_TYPE = '{distance}',
    STANDARDIZE_FEATURES = TRUE,
    num_clusters={n_clusters}) AS (
    (
      select term, geo_name from `{source_table}` where date between '2019-01-01' and '2021-12-31' and category_id = 10047
      order by category_rank asc limit {top_n_results}
    ))
        """).result()

In [141]:
bq_client.query( 
f""" 
 CREATE TABLE IF NOT EXISTS {source_table}_n_clus_{n_clusters}_skincare as (
 SELECT * 
 FROM 
 ML.PREDICT( MODEL trendspotting.{k_means_model_name}, 
 (
    select term from `{source_table}` where date between '2019-01-01' and '2021-12-31' and category_id = 10047
      order by category_rank asc limit {top_n_results}
 )
 ))
     """
).result()

## Create table for all clusters

In [ ]:
bq_client.query( 
f""" 
 CREATE TABLE IF NOT EXISTS {source_table}_n_clus_{n_clusters}_skincare_all as (
 SELECT * 
 FROM 
 ML.PREDICT( MODEL trendspotting.{k_means_model_name}, 
 (
    select * from `{source_table}` where date between '2019-01-01' and '2021-12-31' and category_id = 10047
 )
 ))
     """
).result()

## Create forecast table based on new clusters

In [148]:
bq_client.query( 
f""" 
 CREATE OR REPLACE TABLE {source_table}_n_clus_{n_clusters}_skincare_all_agg as (
  with strat as (select *, 
  case when date between '2019-01-01' and  '2020-12-31' then 'TRAIN'
          when date between '2021-01-01' and '2021-05-31' then 'VALIDATE'
         else 'TEST' end as split_col
   from {source_table}_n_clus_{n_clusters}_skincare_all
  )
  select date, centroid_id, split_col, sum(volume) as volume
    from strat
  group by date, centroid_id, split_col
)
     """
).result()